In [7]:
from dtsp.dataset import walk_forward_split, Seq2SeqDataSet, SimpleSeq2SeqDataSet, SimpleWaveNetDataSet
from dtsp.models import Seq2Seq, SimpleSeq2Seq, SimpleWaveNet
from torch.utils.data import DataLoader
import torch
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [5]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [9]:
df = pd.read_csv('./data.csv', index_col='date_time', parse_dates=['date_time'])

In [14]:
series = df.values
month = df.index.month.values

In [15]:
df.shape

(239, 20)

In [16]:
idxes = list(range(len(series)))

In [12]:
enc_lens = 36
dec_lens = 12
n_test = 12
n_valid = 12

In [18]:
train_idxes, _idxes = walk_forward_split(idxes, enc_lens, dec_lens, test_size=n_test+n_valid)
valid_idxes, test_idxes = walk_forward_split(_idxes, enc_lens, dec_lens, test_size=n_test)

In [19]:
len(train_idxes)

215

In [20]:
len(valid_idxes)

59

In [21]:
len(test_idxes)

59